In [2]:
from konlpy.tag import Twitter
from ipywidgets import FloatProgress
from IPython.display import display
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import pickle
import torch
import os

twitter = Twitter()
if not os.path.exists("./data"):
    os.mkdir("./data")

In [3]:
with open("data/data_deep.pkl", "rb") as f:
    index2voca = pickle.load(f)
    voca2index = pickle.load(f)
    train_X = pickle.load(f)
    train_y = pickle.load(f)
    test_X = pickle.load(f)
    test_y = pickle.load(f)

vocaNum = len(index2voca)
trainNum = len(train_y)
testNum = len(test_y)

embedding_dim = 200
epoch = 10
batch_size = 32
num_iter = int(trainNum/batch_size)
print_iter = 2000
print(f"num_iter: {num_iter}")

num_iter: 7871


In [6]:
train_X = torch.from_numpy(train_X).long()
train_y = torch.from_numpy(train_y).view(-1, 1)
test_X = torch.from_numpy(test_X).long()
test_y = torch.from_numpy(test_y).view(-1, 1)

train = TensorDataset(train_X, train_y)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, num_workers=1)
test = TensorDataset(test_X, test_y)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

mode = 'CPU'

In [4]:
class CNNReg(nn.Module):
    def __init__(self):
        super(CNNReg, self).__init__()
        self.kernel_size = [2,3,4]
        self.channel_out = 40
        self.embedding = nn.Embedding(vocaNum, embedding_dim, padding_idx=0)
        self.conv1 = nn.ModuleList([nn.Conv2d(1, self.channel_out, (k, embedding_dim)) 
                      for k in self.kernel_size])
        self.linear1 = nn.Linear(self.channel_out*len(self.kernel_size),10)
        self.linear2 = nn.Linear(10,1)
        self.dropout = nn.Dropout()
        
    def forward(self, x):
        embed = self.embedding(x)  # (N, W, D)
        embed = embed.unsqueeze(1) # (N,1,W,D), 1: channel_in
        
        #[(N,Channel_out,W,1), ...] * len(Kernel_size)
        feature_maps = [F.leaky_relu(conv(embed)) for conv in self.conv1]
        #[(N,Channel_out,W), ...] * len(Kernel_size)
        feature_maps = [feature_map.squeeze(3) for feature_map in feature_maps]
        
        #[(N, Channel_out), ...] * len(Kernel_size)
        pooled_output = [F.max_pool1d(feature_map, feature_map.size(2)) for feature_map in feature_maps]
        output = torch.cat(pooled_output, 1)
        output = output.view(output.size(0),-1)
        output = self.dropout(output)
        output = F.relu(self.linear1(output))
        output = self.dropout(output)
        output = self.linear2(output)
        return output

In [6]:
reg = CNNReg()
if mode == 'GPU':
    reg.cuda()
print(reg)

criterion = nn.MSELoss()
opt = torch.optim.Adam(reg.parameters())

CNNReg (
  (embedding): Embedding(20000, 200, padding_idx=0)
  (conv1): ModuleList (
    (0): Conv2d(1, 40, kernel_size=(2, 200), stride=(1, 1))
    (1): Conv2d(1, 40, kernel_size=(3, 200), stride=(1, 1))
    (2): Conv2d(1, 40, kernel_size=(4, 200), stride=(1, 1))
  )
  (linear1): Linear (120 -> 10)
  (linear2): Linear (10 -> 1)
  (dropout): Dropout (p = 0.5)
)


In [ ]:
f = FloatProgress(min=0, max=print_iter)
display(f)
for e in range(epoch):
    for i, (batch_X, batch_y) in enumerate(train_loader):
        f.value += 1
        
        if mode == 'GPU':
            batch_X = Variable(batch_X).cuda()
            batch_y = Variable(batch_y).cuda()
        else:
            batch_X = Variable(batch_X)
            batch_y = Variable(batch_y)

        predict = reg(batch_X)
        
        opt.zero_grad()
        loss = criterion(predict, batch_y)
        loss.backward()
        opt.step()
        
        if i%print_iter == 0 or i==num_iter-1:
            f.value=0
            print("batch: {}, iteration: {}, loss: {}\n".format(e, i, loss.data.mean()))
            
torch.save(reg.state_dict(), './data/cnn_regression.pkl')

In [107]:
reg.eval()
criterion = nn.L1Loss()
num_iter = int(testNum/batch_size)
f = FloatProgress(min=0, max=num_iter)
display(f)
average_loss = 0
for i, (batch_X, batch_y) in enumerate(test_loader):
    f.value += 1
    
    if mode == 'GPU':
        batch_X = Variable(batch_X).cuda()
        batch_y = Variable(batch_y).cuda()
    else:
        batch_X = Variable(batch_X)
        batch_y = Variable(batch_y)

    predict = reg(batch_X)
    loss = criterion(predict, batch_y)
    average_loss += loss.data.mean()

average_loss /= num_iter
print("test error(MAE): {}".format(average_loss))
# test error(MAE): 0.17213349469312872

test error(MAE): 0.17213349469312872


In [8]:
reg = CNNReg()
if mode == 'CPU':
    reg.load_state_dict(torch.load("data/cnn_regression.pkl", map_location=lambda storage, loc: storage))
else:
    reg.cuda()
    reg.load_state_dict(torch.load("data/cnn_regression.pkl"))

In [10]:
reg.eval()
test_seq = u"정말 재미 있다."
test_vec = torch.zeros(1, train_X.shape[1]).long()
for i, w in enumerate(twitter.morphs(test_seq)):
    test_vec[0, i] = voca2index.get(w, voca2index['<UNK>'])

test_vec = Variable(test_vec)
predict = reg(test_vec)
print("predicted: {}".format(predict.data.tolist()[0][0]))

predicted: 0.6114215850830078


In [13]:
reg.eval()
test_seq = u"정말 재미 있지 않다."
test_vec = torch.zeros(1, train_X.shape[1]).long()
for i, w in enumerate(twitter.morphs(test_seq)):
    test_vec[0, i] = voca2index.get(w, voca2index['<UNK>'])

test_vec = Variable(test_vec)
predict = reg(test_vec)
print("predicted: {}".format(predict.data.tolist()[0][0]))

predicted: 0.562466025352478
